In [108]:
import pandas as pd
import numpy as np
import pickle
import os
import joblib

from keras.models import Sequential
from tensorflow.keras.regularizers import L1, L2
from keras.layers import Dropout
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from Funciones_tablas_validacion import *


from sklearn.model_selection import GridSearchCV

#Funciones que me paso Emmanuel

from sklearn.feature_selection import SelectFpr, SelectKBest, f_classif, chi2 
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

#### Aplicar features selection a la base, smothe y partición de entrenamiento y Test

In [109]:
#   SUBIMOS LA BASE CON WOE PARA APLICAR UN FEATURE SELECTION Y UN SMOTE
df_woe = pd.read_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/LM (pastilla azul)/Primer Modelo LM/BASE_CN_WOE_S1.csv")
#df_woe.rename(columns={'BGI.1.':'BGI'}, inplace = True)
df_woe.drop(df_woe.filter(regex='^Unnamed:',axis=1),1,inplace=True)
df_woe.head()

,idconsulta,vOpenned,iCredId,vAfiliado,vBanco,BGI,nombreOtorgante_1_AYC_001_Sum_woe,nombreOtorgante_13_AYC_001_Sum_woe,nombreOtorgante_1_AYC_002_Sum_woe,nombreOtorgante_13_AYC_002_Sum_woe,...,tipoContrato_1_INT_128_MOR_Max_woe,tipoContrato_1_INT_129_MOR_Max_woe,tipoContrato_1_INT_130_MOR_Max_woe,tipoContrato_1_INT_131_MOR_Max_woe,tipoContrato_1_INT_132_MOR_Max_woe,tipoContrato_5_INT_136_MOR_Max_woe,tipoContrato_5_INT_137_MOR_Max_woe,tipoContrato_5_INT_138_MOR_Max_woe,tipoContrato_4_INT_145_MOR_Max_woe,tipoContrato_1_INT_148_MOR_Max_woe
0,275935,2018-10-19,305285,IMSS ACTIVOS,BBVA BANCOMER,0,0.226191,-0.236115,0.229986,-0.182163,...,-0.309806,-0.299088,-0.357068,-0.293695,-0.247523,-0.245581,-0.292936,-0.282447,-0.077246,0.064184
1,279475,2018-10-03,300961,IMSS ACTIVOS,BANORTE,0,0.061693,0.271877,0.059065,0.246356,...,0.077319,0.090264,0.121628,0.120579,0.123151,0.099231,0.112603,0.105197,-0.077246,0.064184
2,282291,2018-08-01,278891,IMSS ACTIVOS,SANTANDER,1,-0.440688,-0.238906,-0.440688,-0.238906,...,0.077319,0.090264,0.121628,-0.293695,-0.247523,0.099231,0.112603,0.105197,-0.077246,0.064184
3,282458,2018-08-02,279628,IMSS ACTIVOS,BBVA BANCOMER,0,0.061693,-0.236115,0.059065,-0.182163,...,-0.309806,-0.299088,-0.357068,-0.293695,-0.247523,0.099231,0.112603,0.105197,-0.199715,0.064184
4,282494,2018-09-24,296047,IMSS ACTIVOS,HSBC,0,0.226191,-0.238906,0.229986,-0.238906,...,0.077319,0.090264,0.121628,0.120579,0.123151,0.099231,0.112603,0.105197,0.248620,0.064184


In [110]:
df_woe.shape

(968, 1257)

In [111]:
df_woe.groupby("BGI")["BGI"].count()

BGI
0    739
1    229
Name: BGI, dtype: int64

In [113]:
p_train = 0.60 # Porcentaje de train.

df_woe['is_train'] = np.random.uniform(0, 1, len(df_woe)) <= p_train
#train, train = df_woe[df_woe['is_train']==True], df_woe[df_woe['is_train']==False]
#df_woe = df_woe.drop('is_train', 1)


In [114]:
X_woe = df_woe.iloc[:,6:]
y_woe = df_woe['BGI']

# define the selector to obtain the best 35 features
selector = SelectKBest(f_classif, k=35)
selector.fit(X_woe,y_woe)
new_features = selector.get_feature_names_out()

In [115]:
#CHECAR SI YA GENERAMOS LA DATA O SI ES LA PRIMERA VEZ QUE LA VAMOS A GENERAR
#df_1 = df_woe[['idconsulta','id.credit','RFC.10' ,'fecha_consulta','vOpenned'  ,"BGI","is_train"]+ list(new_features)]
df_1 = df_woe[["idconsulta","vOpenned","iCredId","vAfiliado","vBanco","BGI","is_train"]+ list(new_features)]

#df_1 = pd.read_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/IBILLETE/Primer Modelo Ibillete 14-06-2022/ROBERTO/ROBERTO_BASE_MODELAR_S2.csv")


In [116]:
df_1.head(1)

,idconsulta,vOpenned,iCredId,vAfiliado,vBanco,BGI,is_train,END_002_Sum_woe,END_002_Max_woe,nombreOtorgante_1_END_001_Sum_woe,...,nombreOtorgante_1_INT_082_MOR_Sum_woe,nombreOtorgante_1_INT_083_MOR_Sum_woe,nombreOtorgante_1_INT_084_MOR_Sum_woe,nombreOtorgante_1_INT_087_MOR_Sum_woe,nombreOtorgante_1_INT_089_MOR_Sum_woe,nombreOtorgante_1_INT_090_MOR_Sum_woe,nombreOtorgante_1_INT_137_MOR_Sum_woe,nombreOtorgante_1_INT_145_MOR_Sum_woe,nombreOtorgante_12_INT_060_MOR_Max_woe,tipoCuenta_3_INT_051_MOR_Sum_woe
0,275935,2018-10-19,305285,IMSS ACTIVOS,BBVA BANCOMER,0,True,0.100402,0.096935,0.077859,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,-0.008425,0.479826


In [117]:
# # define the new features
# X_new = df_woe[new_features]
# y_new = y_woe

# #apply SMOTE algoritm
# X_res, y_res = SMOTE(random_state=42).fit_resample(X_new, y_new)


# define the new features
X_new = df_woe[['idconsulta','iCredId' ,"is_train"]+ list(new_features)]
y_new = y_woe

#apply SMOTE algoritm
X_res, y_res = SMOTE(random_state=42).fit_resample(X_new, y_new)




In [118]:
# print(df_woe.groupby("BGI")["BGI"].count())
# print(y_woe.sum())
# print(y_woe.count())
# print(y_res.sum())
# print(y_res.count())
print(X_new.shape)
print(X_res.shape)
print(y_res.shape)


(968, 38)
(1478, 38)
(1478,)


In [ ]:
#df_1 = df_woe[['idconsulta', 'id.credit',"BGI"] + list(new_features)] #take only selected features and target


In [119]:
#print(df_1["BGI"].sum())
#print(df_1["BGI"].count())
print(y_res.sum())
print(y_res.count())

739
1478


In [ ]:
## BASE CON SMOTE
# df = pd.read_csv("BASE_MODELAR_S2_S.csv",encoding ='latin-1')
# df.rename(columns={'BGI.1.':'BGI'}, inplace = True)
# df.head()

In [120]:
df = pd.concat([X_res,y_res],axis=1)
#df= pd.read_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/IBILLETE/Primer Modelo Ibillete 14-06-2022/ROBERTO/ROBERTO_BASE_MODELAR_S2_S.csv")

In [121]:
df.shape

(1478, 39)

In [122]:
df.head(2)


,idconsulta,iCredId,is_train,END_002_Sum_woe,END_002_Max_woe,nombreOtorgante_1_END_001_Sum_woe,nombreOtorgante_1_END_002_Sum_woe,nombreOtorgante_1_END_002_Min_woe,nombreOtorgante_1_NUM_016_BUR_Mean_woe,SDO_002_BUR_Sum_woe,...,nombreOtorgante_1_INT_083_MOR_Sum_woe,nombreOtorgante_1_INT_084_MOR_Sum_woe,nombreOtorgante_1_INT_087_MOR_Sum_woe,nombreOtorgante_1_INT_089_MOR_Sum_woe,nombreOtorgante_1_INT_090_MOR_Sum_woe,nombreOtorgante_1_INT_137_MOR_Sum_woe,nombreOtorgante_1_INT_145_MOR_Sum_woe,nombreOtorgante_12_INT_060_MOR_Max_woe,tipoCuenta_3_INT_051_MOR_Sum_woe,BGI
0,275935,305285,True,0.100402,0.096935,0.077859,0.125213,0.071286,-0.264855,0.025372,...,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,-0.008425,0.479826,0
1,279475,300961,False,0.100402,0.096935,-0.159171,0.125213,0.071286,0.182337,0.597404,...,0.600765,0.700226,0.700226,0.620184,0.635221,0.713402,0.564392,-0.008425,-0.583789,0


In [123]:
#df_1.to_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/LM (pastilla azul)/Primer Modelo LM/BASE_MODELAR_S1.csv")
#df.to_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/LM (pastilla azul)/Primer Modelo LM/BASE_MODELAR_S1_S.csv")

### Hasta aquí el proceso de features selection, Smote, segmentación de entrenamiento y test

#### Si ya estaban las bases solo las cargamos 

In [ ]:
## BASE CON PARTICION
df_1 = pd.read_csv("BASE_MODELAR_S1.csv",encoding = 'latin-1')
df_1.rename(columns={'BGI.1.':'BGI'}, inplace = True)
#df_1.head()
df_1.groupby("BGI")["BGI"].count()

In [ ]:
## BASE CON SMOTE
df = pd.read_csv("BASE_MODELAR_S1_S.csv",encoding = 'latin-1')
df.rename(columns={'BGI.1.':'BGI'}, inplace = True)
df.head()

In [124]:
### TRAIN Y TEST GENERADOS CON BASE SMOTE
df_train = df['is_train'] == True
df_train = df[df_train]

#df_train = df['Partición'] == "1_Entrenamiento"
#df_train = df[df_train]

In [127]:
df_train.head(2)

,idconsulta,iCredId,is_train,END_002_Sum_woe,END_002_Max_woe,nombreOtorgante_1_END_001_Sum_woe,nombreOtorgante_1_END_002_Sum_woe,nombreOtorgante_1_END_002_Min_woe,nombreOtorgante_1_NUM_016_BUR_Mean_woe,SDO_002_BUR_Sum_woe,...,nombreOtorgante_1_INT_083_MOR_Sum_woe,nombreOtorgante_1_INT_084_MOR_Sum_woe,nombreOtorgante_1_INT_087_MOR_Sum_woe,nombreOtorgante_1_INT_089_MOR_Sum_woe,nombreOtorgante_1_INT_090_MOR_Sum_woe,nombreOtorgante_1_INT_137_MOR_Sum_woe,nombreOtorgante_1_INT_145_MOR_Sum_woe,nombreOtorgante_12_INT_060_MOR_Max_woe,tipoCuenta_3_INT_051_MOR_Sum_woe,BGI
0,275935,305285,True,0.100402,0.096935,0.077859,0.125213,0.071286,-0.264855,0.025372,...,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,-0.008425,0.479826,0
2,282291,278891,True,0.100402,0.096935,-0.159171,-0.121311,0.071286,0.182337,-0.441091,...,0.600765,0.700226,0.700226,0.620184,0.635221,0.713402,0.564392,0.299712,0.064296,1


In [126]:
df_train.iloc[:,3:-1]


,END_002_Sum_woe,END_002_Max_woe,nombreOtorgante_1_END_001_Sum_woe,nombreOtorgante_1_END_002_Sum_woe,nombreOtorgante_1_END_002_Min_woe,nombreOtorgante_1_NUM_016_BUR_Mean_woe,SDO_002_BUR_Sum_woe,SDO_002_BUR_Max_woe,nombreOtorgante_1_SDO_002_BUR_Sum_woe,nombreOtorgante_1_SDO_004_BUR_Sum_woe,...,nombreOtorgante_1_INT_082_MOR_Sum_woe,nombreOtorgante_1_INT_083_MOR_Sum_woe,nombreOtorgante_1_INT_084_MOR_Sum_woe,nombreOtorgante_1_INT_087_MOR_Sum_woe,nombreOtorgante_1_INT_089_MOR_Sum_woe,nombreOtorgante_1_INT_090_MOR_Sum_woe,nombreOtorgante_1_INT_137_MOR_Sum_woe,nombreOtorgante_1_INT_145_MOR_Sum_woe,nombreOtorgante_12_INT_060_MOR_Max_woe,tipoCuenta_3_INT_051_MOR_Sum_woe
0,0.100402,0.096935,0.077859,0.125213,0.071286,-0.264855,0.025372,0.041902,-0.048671,0.079616,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,-0.008425,0.479826
2,0.100402,0.096935,-0.159171,-0.121311,0.071286,0.182337,-0.441091,0.041902,-0.048671,0.079616,...,0.613992,0.600765,0.700226,0.700226,0.620184,0.635221,0.713402,0.564392,0.299712,0.064296
4,0.100402,0.096935,0.792034,-0.121311,0.071286,0.182337,0.025372,0.041902,0.545493,0.546076,...,0.014428,0.022545,0.030508,-0.466307,0.033042,0.024962,-0.033852,-0.038477,-0.321425,0.064296
5,0.100402,0.963128,-0.159171,-0.121311,-0.401468,0.182337,0.025372,0.041902,-0.048671,-0.243309,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,0.299712,0.479826
6,0.100402,0.096935,-0.159171,0.583279,0.071286,0.997478,0.025372,0.041902,-0.409436,0.546076,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,0.299712,0.479826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,0.100402,0.096935,0.423345,0.219538,0.071286,0.182337,-0.215437,0.041902,0.238759,0.138560,...,0.304470,0.302262,0.030508,0.338570,-0.343578,0.024962,-0.309011,-0.442061,-0.020947,0.064296
1473,0.100402,0.096935,-0.159171,0.058674,0.071286,0.182337,-0.315187,0.041902,-0.048671,0.079616,...,-0.395086,-0.494806,-0.674535,-0.340533,-0.535357,0.024962,-0.234742,-0.333129,-0.092907,-0.110629
1474,-0.312245,0.096935,0.354330,0.335996,0.071286,-0.059076,0.025372,0.041902,0.106075,0.294261,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.309011,-0.442061,-0.246477,0.064296
1475,0.100402,0.096935,-0.124567,-0.327621,-0.166227,-0.037016,0.504727,0.041902,-0.532913,0.231640,...,-0.570488,-0.687459,-0.896217,-0.503302,-0.737255,-0.764943,-0.371490,-0.482984,0.299712,-0.040702


In [128]:
variables_del_modelo = list(df_train.iloc[:,3:-1].columns)

In [129]:
X_train = df_train.iloc[:,3:-1].values ## Checar que si sean tus variables
y_train = df_train['BGI'].values


In [132]:
#df_train.iloc[:,3:]
#df_1.head(1)
df_1.iloc[:,7:]

,END_002_Sum_woe,END_002_Max_woe,nombreOtorgante_1_END_001_Sum_woe,nombreOtorgante_1_END_002_Sum_woe,nombreOtorgante_1_END_002_Min_woe,nombreOtorgante_1_NUM_016_BUR_Mean_woe,SDO_002_BUR_Sum_woe,SDO_002_BUR_Max_woe,nombreOtorgante_1_SDO_002_BUR_Sum_woe,nombreOtorgante_1_SDO_004_BUR_Sum_woe,...,nombreOtorgante_1_INT_082_MOR_Sum_woe,nombreOtorgante_1_INT_083_MOR_Sum_woe,nombreOtorgante_1_INT_084_MOR_Sum_woe,nombreOtorgante_1_INT_087_MOR_Sum_woe,nombreOtorgante_1_INT_089_MOR_Sum_woe,nombreOtorgante_1_INT_090_MOR_Sum_woe,nombreOtorgante_1_INT_137_MOR_Sum_woe,nombreOtorgante_1_INT_145_MOR_Sum_woe,nombreOtorgante_12_INT_060_MOR_Max_woe,tipoCuenta_3_INT_051_MOR_Sum_woe
0,0.100402,0.096935,0.077859,0.125213,0.071286,-0.264855,0.025372,0.041902,-0.048671,0.079616,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,-0.008425,0.479826
1,0.100402,0.096935,-0.159171,0.125213,0.071286,0.182337,0.597404,0.041902,-0.048671,0.079616,...,0.613992,0.600765,0.700226,0.700226,0.620184,0.635221,0.713402,0.564392,-0.008425,-0.583789
2,0.100402,0.096935,-0.159171,-0.121311,0.071286,0.182337,-0.441091,0.041902,-0.048671,0.079616,...,0.613992,0.600765,0.700226,0.700226,0.620184,0.635221,0.713402,0.564392,0.299712,0.064296
3,1.279429,0.963128,-0.159171,-0.121311,0.071286,-0.264855,0.597404,1.053048,0.545493,-0.243309,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,0.299712,0.064296
4,0.100402,0.096935,0.792034,-0.121311,0.071286,0.182337,0.025372,0.041902,0.545493,0.546076,...,0.014428,0.022545,0.030508,-0.466307,0.033042,0.024962,-0.033852,-0.038477,-0.321425,0.064296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,0.100402,0.096935,-0.159171,0.125213,0.071286,-0.264855,0.597404,0.041902,-0.048671,0.079616,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,0.299712,-0.583789
964,0.100402,0.096935,-0.159171,0.125213,0.071286,0.182337,0.025372,0.041902,-0.048671,0.079616,...,0.014428,0.022545,0.030508,-0.466307,0.033042,0.024962,-0.309011,-0.442061,0.299712,0.064296
965,0.100402,0.096935,0.077859,-0.121311,0.071286,0.182337,0.597404,0.041902,0.545493,0.546076,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.309011,0.564392,0.299712,-0.583789
966,0.100402,0.096935,-0.159171,0.125213,0.071286,-0.264855,0.025372,0.041902,-0.048671,0.079616,...,0.014428,0.022545,0.030508,-0.000324,0.033042,0.024962,-0.033852,-0.038477,-0.321425,-0.583789


In [133]:
X = df_1.iloc[:,7:].values

In [134]:
df_1["Partición"] = np.where(df_1["is_train"]==True,"1_Entrenamiento","2_Comprobación")
df_1.drop(columns="is_train",inplace = True)

In [135]:
df["Partición"] = np.where(df["is_train"]==True,"1_Entrenamiento","2_Comprobación")
df.drop(columns="is_train",inplace = True)

In [ ]:
#df_1.head()
#df.head()
#print(df_1.shape)
#print(df.shape)



### SVM

In [151]:
from sklearn.svm import SVC

param_grid = {'C': [0.1,8], 'gamma': [0.12,0.001]}#,10,.1,
svm = GridSearchCV(SVC(probability=True, kernel = 'sigmoid'),param_grid,scoring='accuracy',cv = 10, n_jobs = -1)
svm.fit(X_train,y_train)

print(svm.best_score_)
print(svm.best_estimator_)

0.6679326099371788
SVC(C=0.1, gamma=0.12, kernel='sigmoid', probability=True)


In [152]:
predictions = svm.predict_proba(X) #### Solo se califica la base sin smote X
predictions = np.array([ predictions[i][0] for i in range(len(predictions))])
df_1['score1'] = np.around(predictions*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
output_path = 'BASE CALIFICADA_S2_SVM.csv'

#df_1.to_csv(output_path, index = False)

In [153]:
#df_tablas = pd.read_csv(output_path)
N = 5
t_train, t_test = tablas_de_entrenamiento_y_validacion(df_1, N, string_score = 'score1', string_bgi = 'BGI_CONS')

In [154]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,649-921,118,100.0%,107,90.7%,100.0%,11,9.3%,100.0%,9.3%,0.0%,9.7,39.3%
1,493-644,119,79.6%,101,84.9%,75.3%,18,15.1%,92.4%,12.2%,17.0%,5.6,35.5%
2,426-492,108,59.0%,83,76.9%,52.1%,25,23.1%,79.9%,15.7%,27.8%,3.3,26.6%
3,334-425,117,40.3%,84,71.8%,32.9%,33,28.2%,62.5%,18.8%,29.6%,2.5,20.7%
4,17-333,116,20.1%,59,50.9%,13.6%,57,49.1%,39.6%,24.9%,26.0%,1.0,7.9%
5,TOTAL,578,,434,,,144,24.9%,,,29.6%,,29.9%


In [156]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,655-934,76,100.0%,66,86.8%,100.0%,10,13.2%,100.0%,13.2%,0.0%,6.6,36.7%
1,493-648,73,80.5%,71,97.3%,78.4%,2,2.7%,88.2%,8.1%,9.9%,35.5,32.6%
2,426-492,85,61.8%,66,77.6%,55.1%,19,22.4%,85.9%,13.2%,30.8%,3.5,32.6%
3,337-425,77,40.0%,57,74.0%,33.4%,20,26.0%,63.5%,16.4%,30.1%,2.8,20.4%
4,17-333,79,20.3%,45,57.0%,14.8%,34,43.0%,40.0%,21.8%,25.2%,1.3,8.1%
5,TOTAL,390,,305,,,85,21.8%,,,30.8%,,30.4%


In [ ]:
tables_to_excel([t_train,t_test], ['Train','Test'], 'Tablas de validación SVM S2')

In [ ]:
joblib.dump(svm, 'SVM_S2.pkl')
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('SVM_S2.pkl')

### Regresión Logística

In [157]:
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.1,1,15,20]}
logreg_cv = GridSearchCV(LogisticRegression(solver = 'liblinear',random_state = 12),param_grid,scoring='accuracy',cv = 10, n_jobs = -1)
logreg_cv.fit(X_train, y_train) #logreg_cv.fit(X_train, y_train)


GridSearchCV(cv=10,
             estimator=LogisticRegression(random_state=12, solver='liblinear'),
             n_jobs=-1, param_grid={'C': [0.1, 1, 15, 20]}, scoring='accuracy')

In [158]:
score = logreg_cv.score(df_1[df_1['Partición'] == '2_Comprobación'][variables_del_modelo], df_1[df_1['Partición'] == '2_Comprobación']['BGI'])
print('Test Accuracy Score', score)

Test Accuracy Score 0.6051282051282051


In [159]:
predictions = logreg_cv.predict_proba(X) #### Solo se califica la base sin smote
predictions = np.array([ predictions[i][0] for i in range(len(predictions))])
df_1['score1'] = np.around(predictions*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
output_path = 'C:/Users/JavierJhairtLópezRoj/OneDrive - Exitus Credit/Escritorio/SEP ACTIVOS 2/Tablas de validacion, bases calificadas, modelos aplicados/Bases calificadas/S1/BASE CALIFICADA_S1_RL.csv'

#df_1.to_csv(output_path, index = False)

In [160]:
#df_tablas = pd.read_csv(output_path)
t_train, t_test = tablas_de_entrenamiento_y_validacion(df_1, N, string_score = 'score1', string_bgi = 'BGI_CONS')

In [161]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,647-888,113,100.0%,107,94.7%,100.0%,6,5.3%,100.0%,5.3%,0.0%,17.8,38.3%
1,537-646,117,80.4%,98,83.8%,75.3%,19,16.2%,95.8%,10.9%,20.5%,5.2,36.1%
2,444-535,117,60.2%,93,79.5%,52.8%,24,20.5%,82.6%,14.1%,29.9%,3.9,30.1%
3,331-442,116,40.0%,76,65.5%,31.3%,40,34.5%,66.0%,19.2%,34.6%,1.9,20.9%
4,12-321,115,19.9%,60,52.2%,13.8%,55,47.8%,38.2%,24.9%,24.4%,1.1,7.6%
5,TOTAL,578,,434,,,144,24.9%,,,34.6%,,33.0%


In [162]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,648-929,79,100.0%,74,93.7%,100.0%,5,6.3%,100.0%,6.3%,0.0%,14.8,39.3%
1,537-646,77,79.7%,66,85.7%,75.7%,11,14.3%,94.1%,10.3%,18.4%,6.0,34.6%
2,449-536,77,60.0%,67,87.0%,54.1%,10,13.0%,81.2%,11.2%,27.1%,6.7,29.7%
3,324-442,78,40.3%,53,67.9%,32.1%,25,32.1%,69.4%,16.4%,37.3%,2.1,21.9%
4,12-321,79,20.3%,45,57.0%,14.8%,34,43.0%,40.0%,21.8%,25.2%,1.3,8.1%
5,TOTAL,390,,305,,,85,21.8%,,,37.3%,,33.6%


In [ ]:
tables_to_excel([t_train,t_test], ['Train','Test'], 'Tablas de validación RL S2')

In [ ]:
joblib.dump(logreg_cv, 'RL_S2.pkl')
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('RL_S2.pkl')

### MLP

In [229]:
from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(alpha=1, hidden_layer_sizes=(2, 4,2), activation = 'identity',
              solver = 'adam', learning_rate = 'adaptive', learning_rate_init = 0.001,random_state=12)
ann.fit(X_train,y_train)

MLPClassifier(activation='identity', alpha=1, hidden_layer_sizes=(2, 4, 2),
              learning_rate='adaptive', random_state=12)

In [249]:
predictions = ann.predict_proba(X) #### Solo se califica la base sin smote
df_1['score1'] = np.around(predictions*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
output_path = 'BASE CALIFICADA_S1_MLP.csv'

df_1.to_csv(output_path, index = False)

In [231]:
#df_tablas = pd.read_csv(output_path)
t_train, t_test = tablas_de_entrenamiento_y_validacion(df_1, N, string_score = 'score1', string_bgi = 'BGI_CONS')

In [232]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,658-928,111,100.0%,105,94.6%,100.0%,6,5.4%,100.0%,5.4%,0.0%,17.5,37.6%
1,530-655,119,80.8%,100,84.0%,75.8%,19,16.0%,95.8%,10.9%,20.0%,5.3,36.7%
2,422-529,116,60.2%,91,78.4%,52.8%,25,21.6%,82.6%,14.5%,29.9%,3.6,29.7%
3,288-420,118,40.1%,79,66.9%,31.8%,39,33.1%,65.3%,19.2%,33.5%,2.0,21.1%
4,7-287,114,19.7%,59,51.8%,13.6%,55,48.2%,38.2%,24.9%,24.6%,1.1,7.5%
5,TOTAL,578,,434,,,144,24.9%,,,33.5%,,32.7%


In [233]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,657-959,81,100.0%,78,96.3%,100.0%,3,3.7%,100.0%,3.7%,0.0%,26.0,40.8%
1,532-655,76,79.2%,64,84.2%,74.4%,12,15.8%,96.5%,9.6%,22.0%,5.3,34.8%
2,421-529,77,59.7%,63,81.8%,53.4%,14,18.2%,82.4%,12.4%,28.9%,4.5,29.3%
3,292-420,76,40.0%,52,68.4%,32.8%,24,31.6%,65.9%,17.1%,33.1%,2.2,20.2%
4,7-286,80,20.5%,48,60.0%,15.7%,32,40.0%,37.6%,21.8%,21.9%,1.5,7.7%
5,TOTAL,390,,305,,,85,21.8%,,,33.1%,,32.8%


In [234]:
tables_to_excel([t_train,t_test], ['Train','Test'], 'Tablas de validación MLP S1')

In [238]:
joblib.dump(ann, 'MLP_S1.pkl')
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('MLP_S1.pkl')

### Red Nueronal Recurrente

In [210]:
#define the keras model
from keras.constraints import max_norm
model = Sequential([
    Dense(8, activation='relu',input_dim= X_train.shape[1]),
    #Dense(10, activation= 'relu', kernel_regularizer= L1(0.1)),
    #model.add(Dropout(0.3))
    #Dropout(0.7),
    #Dense(8, activation='relu', kernel_regularizer= L2(0.1)),
    #Dropout(0.1),
    Dense(190, activation='relu', kernel_regularizer= L2(0.1)),
    Dropout(0.6),
    #Dense(2, activation='relu'),
    #model.add(Dropout(0.3))
    Dense(10, activation='relu'),#, kernel_regularizer= L2()),
    #model.add(Dropout(0.3))
    Dense(2, activation='softmax')
]
)
 
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# fit the keras model on the dataset
history = model.fit(X_train, y_train, epochs=22,
                    validation_data=(df_1[df_1['Partición'] == '2_Comprobación'][variables_del_modelo], df_1[df_1['Partición'] == '2_Comprobación']['BGI']),
                    verbose = 2)

Epoch 1/22
32/32 - 1s - loss: 1.9288 - accuracy: 0.6210 - val_loss: 1.6287 - val_accuracy: 0.4308
Epoch 2/22
32/32 - 0s - loss: 1.3519 - accuracy: 0.6445 - val_loss: 1.2397 - val_accuracy: 0.4333
Epoch 3/22
32/32 - 0s - loss: 1.0131 - accuracy: 0.6621 - val_loss: 1.0066 - val_accuracy: 0.4769
Epoch 4/22
32/32 - 0s - loss: 0.8229 - accuracy: 0.6631 - val_loss: 0.8717 - val_accuracy: 0.4949
Epoch 5/22
32/32 - 0s - loss: 0.7193 - accuracy: 0.6709 - val_loss: 0.7798 - val_accuracy: 0.5154
Epoch 6/22
32/32 - 0s - loss: 0.6676 - accuracy: 0.6885 - val_loss: 0.7664 - val_accuracy: 0.5077
Epoch 7/22
32/32 - 0s - loss: 0.6361 - accuracy: 0.6885 - val_loss: 0.7801 - val_accuracy: 0.4872
Epoch 8/22
32/32 - 0s - loss: 0.6298 - accuracy: 0.6836 - val_loss: 0.7385 - val_accuracy: 0.5333
Epoch 9/22
32/32 - 0s - loss: 0.6140 - accuracy: 0.6964 - val_loss: 0.7593 - val_accuracy: 0.5128
Epoch 10/22
32/32 - 0s - loss: 0.6104 - accuracy: 0.6964 - val_loss: 0.7178 - val_accuracy: 0.5436
Epoch 11/22
32/32 -

In [239]:
output_model_path = 'RNR_M1_S1.h5'#onde #M es el numero del modelo.

#guardar la imagen de cada modelo
model.save(output_model_path)

In [240]:

#Para hacer predicciones sobre nuevos modelos
from keras.models import load_model

model = load_model(output_model_path)

In [241]:
predictions = model.predict(X)
df_1['score1'] = np.around(predictions[:,0]*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
df_1.head()

,idconsulta,vOpenned,iCredId,vAfiliado,vBanco,BGI,END_002_Sum_woe,END_002_Max_woe,nombreOtorgante_1_END_001_Sum_woe,nombreOtorgante_1_END_002_Sum_woe,...,nombreOtorgante_1_INT_089_MOR_Sum_woe,nombreOtorgante_1_INT_090_MOR_Sum_woe,nombreOtorgante_1_INT_137_MOR_Sum_woe,nombreOtorgante_1_INT_145_MOR_Sum_woe,nombreOtorgante_12_INT_060_MOR_Max_woe,tipoCuenta_3_INT_051_MOR_Sum_woe,Partición,score1,BGI_CONS,score1_TILN
0,275935,2018-10-19,305285,IMSS ACTIVOS,BBVA BANCOMER,0,0.100402,0.096935,0.077859,0.125213,...,0.033042,0.024962,-0.033852,-0.038477,-0.008425,0.479826,1_Entrenamiento,542.0,0,3
1,279475,2018-10-03,300961,IMSS ACTIVOS,BANORTE,0,0.100402,0.096935,-0.159171,0.125213,...,0.620184,0.635221,0.713402,0.564392,-0.008425,-0.583789,2_Comprobación,700.0,0,3
2,282291,2018-08-01,278891,IMSS ACTIVOS,SANTANDER,1,0.100402,0.096935,-0.159171,-0.121311,...,0.620184,0.635221,0.713402,0.564392,0.299712,0.064296,1_Entrenamiento,739.0,1,5
3,282458,2018-08-02,279628,IMSS ACTIVOS,BBVA BANCOMER,0,1.279429,0.963128,-0.159171,-0.121311,...,0.033042,0.024962,-0.033852,-0.038477,0.299712,0.064296,2_Comprobación,780.0,0,5
4,282494,2018-09-24,296047,IMSS ACTIVOS,HSBC,0,0.100402,0.096935,0.792034,-0.121311,...,0.033042,0.024962,-0.033852,-0.038477,-0.321425,0.064296,1_Entrenamiento,792.0,0,4


In [242]:
df = df_1

### Tabla de Validación

##### ENTRENAMIENTO

In [243]:
output_path = 'BASE CALIFICADA_S1_RNR_M4.csv'

In [244]:
df_1.to_csv(output_path)

In [245]:
#df_tablas = pd.read_csv(output_path)
N = 5
t_train, t_test = tablas_de_entrenamiento_y_validacion(df,N, string_score = 'score1', string_bgi = 'BGI')


In [246]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,756-886,110,100.0%,102,92.7%,100.0%,8,7.3%,100.0%,7.3%,0.0%,12.8,37.0%
1,663-755,124,81.0%,109,87.9%,76.5%,15,12.1%,94.4%,9.8%,17.9%,7.3,38.3%
2,482-661,124,59.5%,105,84.7%,51.4%,19,15.3%,84.0%,11.7%,32.6%,5.5,33.2%
3,268-476,111,38.1%,76,68.5%,27.2%,35,31.5%,70.8%,16.4%,43.6%,2.2,22.5%
4,44-267,109,18.9%,42,38.5%,9.7%,67,61.5%,46.5%,24.9%,36.9%,0.6,8.8%
5,TOTAL,578,,434,,,144,24.9%,,,43.6%,,39.8%


In [247]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,757-904,83,100.0%,79,95.2%,100.0%,4,4.8%,100.0%,4.8%,0.0%,19.8,41.6%
1,663-752,70,78.7%,62,88.6%,74.1%,8,11.4%,95.3%,7.8%,21.2%,7.8,32.5%
2,486-651,70,60.8%,59,84.3%,53.8%,11,15.7%,85.9%,10.3%,32.1%,5.4,28.5%
3,269-481,82,42.8%,53,64.6%,34.4%,29,35.4%,72.9%,17.0%,38.5%,1.8,23.5%
4,44-267,85,21.8%,52,61.2%,17.0%,33,38.8%,38.8%,21.8%,21.8%,1.6,8.5%
5,TOTAL,390,,305,,,85,21.8%,,,38.5%,,34.5%


In [248]:
tables_to_excel([t_train,t_test],['Train','Test'],'Tablas de validación RNR S1')